## Directed Percolation Simulation Example

This notebook demonstrates how to use the `DirectedPercolation` module to simulate the model and visualize the results.

### Project Setup

To run this notebook, your directory structure should look like this:
```
your_project_folder/
├── src/
│   └── Directed_Percolation.jl
├── example.ipynb
└── Project.toml  (this will be generated by Pkg)
```

First, we need to set up a Julia environment for this project to manage its dependencies (`Plots`).

In [ ]:
using Pkg
# Activate the environment in the current directory
Pkg.activate(".")
# Add the necessary packages for plotting and statistics
Pkg.add(["Plots", "Statistics"])

### Loading the Module and Dependencies

Now, we'll load our custom module and the packages we'll use for plotting and analysis.

In [ ]:
# Include the source file for our module
include("src/Directed_Percolation.jl")
# Bring the module's functions into the current scope
using .DirectedPercolation

# Load external packages
using Plots
using Random
using Statistics

### Visualization Functions

These functions are responsible for running the simulations and plotting the results. They now live in the notebook, as they represent the analysis part of the workflow, while the core simulation logic is in the module.

In [ ]:
"""
    plot_density_vs_time(N::Int, p::Float64, q::Float64, t_max::Int, num_trials::Int)

Runs multiple simulations and plots the average active site density vs. time.
"""
function plot_density_vs_time(N::Int, p::Float64, q::Float64, t_max::Int, num_trials::Int)
    println("Generating plot for p=$p, q=$q...")
    total_density_over_time = zeros(Float64, t_max + 1)

    for _ in 1:num_trials
        initial_state = generate_initial_state(N, density=0.5)
        history = evolve(N, p, q, t_max, initial_state)
        density_over_time = calculate_average_density(history)
        total_density_over_time .+= density_over_time
    end

    avg_density = total_density_over_time / num_trials

    plot(0:t_max, avg_density,
        title="Average Active Site Density vs. Time\n(p=$p, q=$q, N=$N, trials=$num_trials)",
        xlabel="Time (t)",
        ylabel="Average Density",
        label="Density",
        legend=:topright,
        linewidth=2
    )
end

In [ ]:
"""
    plot_phase_diagram(N::Int, t_final::Int, p_steps::Int, q_steps::Int, num_trials::Int)

Generates a 2D color-coded phase diagram of final active site density.
"""
function plot_phase_diagram(N::Int, t_final::Int, p_steps::Int, q_steps::Int, num_trials::Int)
    println("Generating phase diagram...")
    p_range = range(0, 1, length=p_steps)
    q_range = range(0, 1, length=q_steps)
    
    final_densities = zeros(Float64, length(q_range), length(p_range))

    for (j, p) in enumerate(p_range)
        for (i, q) in enumerate(q_range)
            avg_final_density = 0.0
            for _ in 1:num_trials
                initial_state = generate_initial_state(N, density=0.8)
                history = evolve(N, p, q, t_final, initial_state)
                density_over_time = calculate_average_density(history)
                start_index = floor(Int, 0.9 * t_final)
                avg_final_density += mean(density_over_time[start_index:end])
            end
            final_densities[i, j] = avg_final_density / num_trials
        end
        println("Progress: $(round(j/p_steps*100, digits=1))%")
    end

    heatmap(p_range, q_range, final_densities,
        title="Directed Percolation Phase Diagram\n(N=$N, t_final=$t_final, trials=$num_trials)",
        xlabel="p (Probability)",
        ylabel="q (Survival Rate)",
        c=:viridis,
        colorbar_title="Final Active Site Density"
    )
end

### Main Execution

Here, we set the parameters and run the analysis.

In [ ]:
Random.seed!(1234) # for reproducibility

# --- Parameters for the density vs. time plot ---
N_time_plot = 100
p_time_plot = 0.7
q_time_plot = 0.8
t_max_time_plot = 200
num_trials_time_plot = 50

# Generate and display the plot
plot1 = plot_density_vs_time(N_time_plot, p_time_plot, q_time_plot, t_max_time_plot, num_trials_time_plot)
savefig(plot1, "density_vs_time_p$(p_time_plot)_q$(q_time_plot).png")
display(plot1)

In [ ]:
# --- Parameters for the phase diagram ---
N_phase = 50
t_final_phase = 100
p_steps_phase = 25
q_steps_phase = 25
num_trials_phase = 5

# Generate and display the phase diagram
plot2 = plot_phase_diagram(N_phase, t_final_phase, p_steps_phase, q_steps_phase, num_trials_phase)
savefig(plot2, "phase_diagram.png")
display(plot2)